In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Make sure pad_token_id is set properly
pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [ ]:
import pandas as pd

### Inference for Confidence

In [ ]:
import numpy as np
from IPython.display import HTML, display

# Prepare input question, prompt asking directly for the answer

system_prompt = "You are a highly accurate question-answering assistant. Answer concisely with only the best option (A, B, C, or D) and a confidence score (0 to 1), formatted as 'Answer: X, Confidence: Y'."

question = """Question: An important source of information on the credit rating of retail businesses is

A) the Retail Merchants Association
B) the local chamber of commerce
C) Dun & Bradstreet, Inc.
D) the United States Retail Credit Association

Just answer with the best option (A, B, C, or D), do not add anything else to your output:"""

# Tokenize the input question
inputs = tokenizer(system_prompt + "\n\n" + question, return_tensors="pt", padding=True).to(model.device)

# Generate response with limited tokens (for short answers)
with torch.no_grad():
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=8,  # Limit to few tokens
        pad_token_id=pad_token_id,
        return_dict_in_generate=True,
        output_scores=True,
        temperature=1
    )

# Decode the generated text (should only be one letter now)
generated_tokens = outputs.sequences[0]
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
print("Generated answer:", generated_text)

# Extract logits for the generated tokens
logits = outputs.scores

# Calculate log probabilities for each token in the generated response
log_probs = []
linear_probs = []
for i, logits_per_token in enumerate(logits):
    probs = torch.nn.functional.log_softmax(logits_per_token, dim=-1)
    token_id = generated_tokens[i + len(inputs.input_ids[0])]
    log_prob = probs[0, token_id].item()  # Get log probability of the token
    log_probs.append(log_prob)
    
    # Convert log probability to linear probability (percentage)
    linear_prob = np.exp(log_prob) * 100
    linear_probs.append(linear_prob)

# Print logprobs and linear probabilities for each token
print("\nLogprobs and Linear Probabilities for each generated token:")
for token, log_prob, linear_prob in zip(generated_tokens[len(inputs.input_ids[0]):], log_probs, linear_probs):
    token_str = tokenizer.decode([token])
    print(f"Token: {token_str}, Logprob: {log_prob}, Linear Probability: {linear_prob:.2f}%")


### Inference for Logprob only

In [ ]:

def single_inference_llama(question, system_prompt=system_prompt):
    # Tokenize the input question
    inputs = tokenizer(system_prompt + "\n\n" + question, return_tensors="pt", padding=True).to(model.device)
    
    # Generate response with limited tokens (for short answers)
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=5,  # Limit to few tokens
            pad_token_id=pad_token_id,
            return_dict_in_generate=True,
            output_scores=True,
            temperature=1
        )

    # Decode the generated text (should only be one letter now)
    generated_tokens = outputs.sequences[0]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    # print("Generated answer:", generated_text)

    # Extract logits for the generated tokens
    logits = outputs.scores

    # Calculate log probabilities for each token in the generated response
    log_probs = []
    linear_probs = []
    for i, logits_per_token in enumerate(logits):
        probs = torch.nn.functional.log_softmax(logits_per_token, dim=-1)
        token_id = generated_tokens[i + len(inputs.input_ids[0])]
        log_prob = probs[0, token_id].item()  # Get log probability of the token
        log_probs.append(log_prob)
        
        # Convert log probability to linear probability (percentage)
        linear_prob = np.exp(log_prob) * 100
        linear_probs.append(linear_prob)


    return generated_text, log_probs, linear_probs

    # # Print logprobs and linear probabilities for each token
    # print("\nLogprobs and Linear Probabilities for each generated token:")
    # for token, log_prob, linear_prob in zip(generated_tokens[len(inputs.input_ids[0]):], log_probs, linear_probs):
    #     token_str = tokenizer.decode([token])
    #     print(f"Token: {token_str}, Logprob: {log_prob}, Linear Probability: {linear_prob:.2f}%")





In [3]:

import pandas as pd

df = pd.read_csv('MMLU_5000_sample.csv')[:1000]

responses = []

for i, row in df.iterrows():
    question = row['prompt']
    answer = row['answer']

    response = single_inference_llama(question)

    responses.append(response)


new_df = pd.DataFrame(responses, columns=['generated', 'log_probs', 'linear_probs'])
new_df['correct_answer'] = df['answer']

new_df.to_csv("llama3.1_lobprob_benchmark.csv", index=False)


,prompt,answer
0,Question: An important source of information o...,C
1,Question: In preparation for a writing unit on...,B
2,Question: Paper will burn at approximately wha...,D
3,Question: The Apple iMac computer is available...,C
4,Question: What were the first names of the ear...,B
...,...,...
995,Question: For which of these two scenarios doe...,B
996,Question: For which of these two scenarios doe...,A
997,Question: For which of these two scenarios doe...,A
998,Question: For which of these two scenarios doe...,B


In [ ]:
import numpy as np
from IPython.display import HTML, display

# Prepare input question, prompt asking directly for the answer

system_prompt = "You are a highly accurate question-answering assistant. Answer concisely with only the best option (A, B, C, or D) formatted as 'Answer: X'."

question = """Question: An important source of information on the credit rating of retail businesses is

A) the Retail Merchants Association
B) the local chamber of commerce
C) Dun & Bradstreet, Inc.
D) the United States Retail Credit Association

Just answer with the best option (A, B, C, or D), do not add anything else to your output:"""


# Tokenize the input question
inputs = tokenizer(system_prompt + "\n\n" + question, return_tensors="pt", padding=True).to(model.device)

# Generate response with limited tokens (for short answers)
with torch.no_grad():
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=5,  # Limit to few tokens
        pad_token_id=pad_token_id,
        return_dict_in_generate=True,
        output_scores=True,
        temperature=1
    )

# Decode the generated text (should only be one letter now)
generated_tokens = outputs.sequences[0]
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
print("Generated answer:", generated_text)

# Extract logits for the generated tokens
logits = outputs.scores

# Calculate log probabilities for each token in the generated response
log_probs = []
linear_probs = []
for i, logits_per_token in enumerate(logits):
    probs = torch.nn.functional.log_softmax(logits_per_token, dim=-1)
    token_id = generated_tokens[i + len(inputs.input_ids[0])]
    log_prob = probs[0, token_id].item()  # Get log probability of the token
    log_probs.append(log_prob)
    
    # Convert log probability to linear probability (percentage)
    linear_prob = np.exp(log_prob) * 100
    linear_probs.append(linear_prob)

# Print logprobs and linear probabilities for each token
print("\nLogprobs and Linear Probabilities for each generated token:")
for token, log_prob, linear_prob in zip(generated_tokens[len(inputs.input_ids[0]):], log_probs, linear_probs):
    token_str = tokenizer.decode([token])
    print(f"Token: {token_str}, Logprob: {log_prob}, Linear Probability: {linear_prob:.2f}%")
